In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, BasicAer, transpile
from math import pi, sqrt

In [2]:
def inv_qft_gate(nbdigits):
    qr = QuantumRegister(nbdigits, 'qr')
    circuit = QuantumCircuit(qr)
    for i in range(nbdigits // 2):
        circuit.swap(i, nbdigits-i-1)
    for i in range(nbdigits):
        circuit.h(qr[i])
        for j in range(i+1, nbdigits):
            rot_phase = - 2*pi/2**(j-i+1)
            circuit.cp(rot_phase, j, i)
    return circuit.to_gate()

def phase_estimation_circuit(gate, nbdigits, teststate_nbdigits, initial_monopartite_state=np.array([0, 1])):
    # customized controlled gate
    controlled_gate = gate.control(1)

    # initial_state
    assert len(initial_monopartite_state) == 2**teststate_nbdigits
    assert np.linalg.norm(initial_monopartite_state) == 1
    initial_state = np.zeros(2 ** (nbdigits + teststate_nbdigits), dtype=np.complex_)
    for i in range(2 ** teststate_nbdigits):
        initial_state[i * 2 ** nbdigits] = initial_monopartite_state[i]

    # building the circuit
    circuit = QuantumCircuit(nbdigits + teststate_nbdigits, nbdigits)
    circuit.initialize(initial_state)
    circuit.h(range(nbdigits))
    for i in range(nbdigits):
        for _ in range(2 ** (i)):
            circuit.append(controlled_gate, [i]+[j+nbdigits for j in range(teststate_nbdigits)])
    circuit.append(inv_qft_gate(nbdigits), range(nbdigits))
    circuit.measure(range(nbdigits), range(nbdigits))

    return circuit

In [3]:
nbdigits = 5

In [4]:
from qiskit.circuit.library import PhaseGate

theta = 0.5+0.25
teststate_nbdigits = 2

gate_circuit = QuantumCircuit(teststate_nbdigits)
for i in range(teststate_nbdigits):
    gate_circuit.p(2*pi*theta*i, i)
gate = gate_circuit.to_gate()

In [5]:
initial_monopartite_state = np.zeros(2**teststate_nbdigits)
initial_monopartite_state[2] = 1

In [6]:
circuit = phase_estimation_circuit(gate, nbdigits, teststate_nbdigits, initial_monopartite_state=initial_monopartite_state)

In [7]:
backend = Aer.get_backend('aer_simulator')
tqc = transpile(circuit, backend)
job = backend.run(tqc, shots=1000)
result = job.result()
counts = result.get_counts()
print(counts)

{'11000': 1000}
